In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
#
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
credit = pd.read_csv("C:\\bbk\predictiveanalytics\course2\DefaultedCreditCardClients.csv")

In [3]:
credit.head()

,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,AGE_GROUP,default,LIMIT_BAL_GROUP,LIMIT_BAL
0,0,0,0,689,0,0,0,0,24,1,1,1,20000
1,0,3455,0,1000,1000,1000,0,2000,26,2,1,3,120000
2,0,14948,1518,1500,1000,1000,1000,5000,34,3,0,2,90000
3,0,28959,2000,2019,1200,1100,1069,1000,37,4,0,1,50000
4,0,19146,2000,36681,10000,9000,689,679,57,7,0,1,50000


In [4]:
#remove unnecessary data that does not make any sense
#drop duplicates - always
credit = credit.drop_duplicates()
#check for the nulls
credit.dropna(axis=0, how="any")
nulls = credit.isnull().sum()
nulls

GRADUATE           0
BILL_AMT5          0
PAY_AMT1           0
PAY_AMT2           0
PAY_AMT3           0
PAY_AMT4           0
PAY_AMT5           0
PAY_AMT6           0
AGE                0
AGE_GROUP          0
default            0
LIMIT_BAL_GROUP    0
LIMIT_BAL          0
dtype: int64

In [5]:
credit.dtypes

GRADUATE           int64
BILL_AMT5          int64
PAY_AMT1           int64
PAY_AMT2           int64
PAY_AMT3           int64
PAY_AMT4           int64
PAY_AMT5           int64
PAY_AMT6           int64
AGE                int64
AGE_GROUP          int64
default            int64
LIMIT_BAL_GROUP    int64
LIMIT_BAL          int64
dtype: object

In [6]:
credit['default'].describe()

count    29673.000000
mean         0.220807
std          0.414797
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default, dtype: float64

In [7]:
credit.describe()
#Let's review some of the data analysis output from the DataFrame describe method
# 1. Sanity Check: All counts are same at 29965

,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,AGE_GROUP,default,LIMIT_BAL_GROUP,LIMIT_BAL
count,29673.000000,29673.000000,29673.000000,2.967300e+04,29673.000000,29673.000000,29673.000000,29673.000000,29673.000000,29673.000000,29673.000000,29673.000000,29673.000000
mean,0.351296,40753.744414,5725.735180,5.986292e+03,5283.145284,4879.136960,4852.153608,5272.854177,35.501870,3.462811,0.220807,3.257305,167330.896101
std,0.477383,60984.200602,16643.644619,2.315908e+04,17695.153072,15744.043323,15353.942551,17866.709541,9.232184,1.791121,0.414797,1.827954,129876.644855
min,0.000000,-81334.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,21.000000,1.000000,0.000000,1.000000,10000.000000
25%,0.000000,1981.000000,1000.000000,1.000000e+03,444.000000,324.000000,315.000000,200.000000,28.000000,2.000000,0.000000,1.000000,50000.000000
50%,0.000000,18413.000000,2157.000000,2.037000e+03,1880.000000,1502.000000,1544.000000,1500.000000,34.000000,3.000000,0.000000,3.000000,140000.000000
75%,1.000000,50673.000000,5025.000000,5.000000e+03,4600.000000,4100.000000,4100.000000,4032.000000,41.000000,5.000000,0.000000,5.000000,240000.000000
max,1.000000,927171.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,79.000000,8.000000,1.000000,7.000000,1000000.000000


In [8]:
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
#**************************************************************
#       Course2 Task 3 - Now Let us build the Model
#**************************************************************

In [10]:
################################################################################
# Course2 Task3 - Juptyer Notebook worksheet #
# This Block makes all the Tools required to be Available for use
# Import Basic Tools for DS dat analysis
# Note: you may have to install pandas_profiling package before this could be run, if not installed already
# conda activate datasciencebbk
# conda install -c conda-forge pandas-profiling
################################################################################
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt


# Import SKLearn package
from sklearn.model_selection import train_test_split, cross_val_score, KFold, LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Import SKLearn model metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

# Helper classes
%matplotlib inline

In [11]:
credit.head()

,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,AGE_GROUP,default,LIMIT_BAL_GROUP,LIMIT_BAL
0,0,0,0,689,0,0,0,0,24,1,1,1,20000
1,0,3455,0,1000,1000,1000,0,2000,26,2,1,3,120000
2,0,14948,1518,1500,1000,1000,1000,5000,34,3,0,2,90000
3,0,28959,2000,2019,1200,1100,1069,1000,37,4,0,1,50000
4,0,19146,2000,36681,10000,9000,689,679,57,7,0,1,50000


In [12]:
# Indepedent variables assignement to X
X = credit.iloc[:, [0,1,2,3,4,5,6,7,8,12]]
print('----------------------------------------------------')
print('Summary of Independent variable Samples')
X.head()


----------------------------------------------------
Summary of Independent variable Samples


,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,LIMIT_BAL
0,0,0,0,689,0,0,0,0,24,20000
1,0,3455,0,1000,1000,1000,0,2000,26,120000
2,0,14948,1518,1500,1000,1000,1000,5000,34,90000
3,0,28959,2000,2019,1200,1100,1069,1000,37,50000
4,0,19146,2000,36681,10000,9000,689,679,57,50000


In [13]:
print('----------------------------------------------------')
# Lets assign Outcome (dependent variable) for Y-axis - LIMIT_BAL_GROUP
y = credit['default']
y.head()

----------------------------------------------------


0    1
1    1
2    0
3    0
4    0
Name: default, dtype: int64

In [14]:
# Decide and select the models for comparison and loop them in cross_val_score by passing appropriate model
# algo_class is an array where we will assign the model name and the ModelClassifier
algos_Class = []
algos_Class.append(('Support Vector Classifier', SVC(gamma="auto")))# SVC classification
algos_Class.append(('Random Forest Classifier', RandomForestClassifier()))
algos_Class.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))
algos_Class.append(('Decision Tree Classifier', DecisionTreeClassifier()))
algos_Class.append(('K Neighbors Classifier', KNeighborsClassifier(n_neighbors=1)))#classification
results = []
names = []
for name, model in algos_Class:
    result = cross_val_score(model, X,y, cv=3, scoring='accuracy')
    names.append(name)
    results.append(result)
#output
for i in range(len(names)):
    print(names[i],results[i].mean())

Support Vector Classifier 0.7780136824722811
Random Forest Classifier 0.7805412327705321
Gradient Boosting Classifier 0.7862703467799009
Decision Tree Classifier 0.6843932194250666
K Neighbors Classifier 0.6930542917804065


In [15]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)
X_train.head()
y_train.head()
y_test.head()

23849    0
4758     0
19055    1
2662     0
14935    0
Name: default, dtype: int64

In [16]:
# Based on the comparison above, Using the LIMIT_BAL_GROUP produces more accuracy
# However, there risk is associated with Discretization Range selected for the LIMIT_BAL_GROUP.
# We will use GradientBoosting Classifer and predict
# Then we will use the RandomForestClassifer and predict.
algo = GradientBoostingClassifier(max_depth=4)
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.80      0.97      0.88      6943
           1       0.58      0.15      0.24      1959

    accuracy                           0.79      8902
   macro avg       0.69      0.56      0.56      8902
weighted avg       0.75      0.79      0.74      8902



In [17]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for Gradient Boosting Classifier :')
print(results)

Confusion Matrix for Gradient Boosting Classifier :
[[6725  218]
 [1662  297]]


In [18]:
#Train/Test Split for RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)
X_train.head()
y_train.head()
y_test.describe()

count    8902.000000
mean        0.220063
std         0.414312
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: default, dtype: float64

In [19]:
# Based on the comparison above, Using the LIMIT_BAL_GROUP produces more accuracy when compared with LIMIT_BAL itself
# However, the risk is associated with Discretization Range selected for the LIMIT_BAL_GROUP.
#Modeling (Classification)
algo = RandomForestClassifier(max_depth=4)
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      6943
           1       0.00      0.00      0.00      1959

    accuracy                           0.78      8902
   macro avg       0.39      0.50      0.44      8902
weighted avg       0.61      0.78      0.68      8902



In [20]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for RandomForestClassifier:')
print(results)

Confusion Matrix for RandomForestClassifier:
[[6941    2]
 [1959    0]]


In [21]:
#Modeling (Classification - KNeighbours classifier)
algo = KNeighborsClassifier(n_neighbors=1)
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.80      0.81      0.80      6943
           1       0.29      0.27      0.28      1959

    accuracy                           0.69      8902
   macro avg       0.54      0.54      0.54      8902
weighted avg       0.68      0.69      0.69      8902



In [22]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for K Neighbours Classifier:')
print(results)

Confusion Matrix for K Neighbours Classifier:
[[5643 1300]
 [1436  523]]


In [23]:
#Modeling (Classification - Decision Tree classifier)
algo = DecisionTreeClassifier()
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80      6943
           1       0.30      0.33      0.31      1959

    accuracy                           0.69      8902
   macro avg       0.56      0.56      0.56      8902
weighted avg       0.70      0.69      0.69      8902



In [24]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for Decision Tree Classifier:')
print(results)

Confusion Matrix for Decision Tree Classifier:
[[5486 1457]
 [1321  638]]


In [25]:
from sklearn.svm import SVC
#Modeling (Classification - Support Vector classifier)
algo = SVC(gamma="auto")
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      6943
           1       0.32      0.01      0.01      1959

    accuracy                           0.78      8902
   macro avg       0.55      0.50      0.44      8902
weighted avg       0.68      0.78      0.69      8902



In [26]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for Support Vector Classifier:')
print(results)

Confusion Matrix for Support Vector Classifier:
[[6920   23]
 [1948   11]]
